In [ ]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.image as mpimg
import cv2 as cv
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ELU
from keras.layers import Lambda, Dense, Dropout, Flatten
from keras.initializers import RandomUniform
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import ELU
from keras.optimizers import Adam
%matplotlib inline
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# Global Vars
img = (cv.imread('../train/photos/images/gray_Clipped/0.jpg')).shape
img_height = img[0]
img_width = img[1]
# img_channels = img[2]
img_channels = 2
print(img, img_height, img_width, img_channels)

In [ ]:
def adjust_Value(prvs_FRAME, curr_FRAME, HSV_value):
    prvs_FRAME = cv.cvtColor(prvs_FRAME, cv.COLOR_BGR2HSV)
    curr_FRAME = cv.cvtColor(curr_FRAME, cv.COLOR_BGR2HSV)
    prvs_FRAME[...,2] = prvs_FRAME[...,2]*HSV_value
    curr_FRAME[...,2] = curr_FRAME[...,2]*HSV_value
    prvs_FRAME = cv.cvtColor(prvs_FRAME, cv.COLOR_HSV2BGR)
    curr_FRAME = cv.cvtColor(curr_FRAME, cv.COLOR_HSV2BGR)
    return prvs_FRAME, curr_FRAME

In [ ]:
def opticalFlow_DENSE(prvs_FRAME, curr_FRAME, dynamic_sat):
    global img_height, img_width, img_channels
    
    flow_mat = None
    image_scale = 0.5
    pyr_images = 1 # was 3
    win_size = 20
    pyr_iterations = 10 # was 3, 2; 10 seems to be doing well clearing up the image; 20 is too much for little gain
    poly_expans = 5 # was 5; 5 seems to be the best
    std = 5 # was 1.2 was 2.3; higher seems to be better ex:5,10

    #hsv = np.zeros_like(prvs_FRAME)
    hsv = np.zeros((img_height, img_width, img_channels), dtype='float32')
    #if dynamic_sat:
        #hsv[..., 1] = cv.cvtColor(prvs_FRAME, cv.COLOR_BGR2HSV)[...,1]
    #else:
        #hsv[...,1] = 255
    prvs_FRAME = cv.cvtColor(prvs_FRAME,cv.COLOR_BGR2GRAY)
    curr_FRAME = cv.cvtColor(curr_FRAME,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs_FRAME,curr_FRAME,flow_mat,image_scale,pyr_images,win_size,pyr_iterations,poly_expans,std,0)
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*(180/np.pi/2)
    mag[mag>100]= 100
    hsv[...,1] = mag/100
    #hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    #hsv = np.asarray(hsv, dtype=np.float32)
    #bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    #return bgr
    return hsv

In [ ]:
def getData(train_meta, kLoopCount, train_index, doubleSpeed, getSpeed, for_training):
    global img_height, img_width, img_channels
    opt_flows = np.empty((0, img_height, img_width, img_channels), dtype='float32')
    speed = np.empty((0))
    for k in range(kLoopCount):
        HSV_value = np.random.uniform(low=.5, high=1.2)
        #print(train_index)
        curr_FRAME = cv.imread(train_meta['image_path'][train_index])
        if doubleSpeed:
            prvs_FRAME = cv.imread(train_meta['image_path'][train_index-2])
        else:
            prvs_FRAME = cv.imread(train_meta['image_path'][train_index-1])
        if getSpeed:
            speed1 = train_meta['speed'][train_index]
            speed2 = train_meta['speed'][train_index-1]
            speed3 = np.mean((speed1, speed2))
            speed4 = speed1+speed2
            if doubleSpeed:
                speed = np.append(speed, speed4)
            else:
                speed = np.append(speed, speed1)
            #speed = np.append(speed, speed1)
        if for_training:
            prvs_FRAME, curr_FRAME = adjust_Value(prvs_FRAME, curr_FRAME, HSV_value)
        opt_flow = opticalFlow_DENSE(prvs_FRAME, curr_FRAME, False)
#         opt_flows = np.append(opt_flows, [opt_flow], axis=0)
        opt_flows = np.append(opt_flows, [np.asarray(opt_flow, dtype=np.float32)], axis=0)
    if getSpeed:
        return opt_flows, speed
    else:
        return opt_flows

In [ ]:
def getTrainingData(train_meta, train_index, batchSize):
    global img_height, img_width, img_channels#, index_count
    get_speed = True
    for_training = True
    while True:
        opt_flow_arr = np.empty((0, img_height, img_width, img_channels))
        speed_arr = np.empty((0))
        for i in range(batchSize):
            index = np.random.randint(0, train_index.shape[0], dtype='int16')
            doubleSpeed = np.random.randint(0, 10)
            if doubleSpeed < 2:
                doubleSpeed = True
            else:
                doubleSpeed = False
            opt_flow, speed = getData(train_meta, 1, train_index[index], doubleSpeed, get_speed, for_training)
            opt_flow_arr = np.append(opt_flow_arr, opt_flow, axis=0)
            speed_arr = np.append(speed_arr, speed)
        yield opt_flow_arr, speed_arr

In [ ]:
def getValData(train_meta, val_index, getSpeed):
    doubleSpeed = False
    for_training = False
    while True:
        index = 0
        for i in range(len(val_index)):
            if getSpeed:
                opt_flow, speed = getData(train_meta, 1, val_index[index], doubleSpeed, getSpeed, for_training)
            else:
                opt_flow = getData(train_meta, 1, val_index[index], doubleSpeed, getSpeed, for_training)
            #opt_flow_arr = np.append(opt_flow_arr, opt_flow)
            index = index + 1
            if getSpeed:
                yield opt_flow, speed
            else:
                yield opt_flow

In [ ]:
def createCNNModel():
    global img_height, img_width, img_channels, num_layers, activ, reg, learning_rate, epsilon, loss
    global kern_init, reg2, pad, custom_opt
    
    model = Sequential()
    model.add(Lambda(lambda x: x, input_shape=(img_height, img_width, img_channels)))
    #model.add(Lambda(lambda x: x / 127.5 - 1, input_shape=(img_height, img_width, img_channels)))
    #uniform_initializer = RandomUniform(minval=1e-10, maxval=1e-9, seed=1)
    
    model.add(Conv2D(filters=160, 
                     kernel_size=(5,5), 
                     strides=(3,3), 
                     activation=activ))#, padding=pad, kernel_initializer = kern_init, kernel_regularizer=l2(reg2)))
    model.add(MaxPooling2D((10, 10)))
    if num_layers > 1:
        model.add(Conv2D(filters=320, 
                     kernel_size=(5,5), 
                     strides=(3,3), 
                     activation=activ))#, padding=pad, kernel_initializer = kern_init, kernel_regularizer=l2(reg2)))
        model.add(MaxPooling2D((3, 3)))
    if num_layers > 2:
        model.add(Conv2D(filters=640, 
                     kernel_size=(5,5), 
                     strides=(3,3), 
                     activation=activ))#, padding=pad, kernel_initializer = kern_init, kernel_regularizer=l2(reg2)))
        model.add(MaxPooling2D((3, 3)))
    
    model.add(Flatten())
    
    model.add(Dense(units=128, activation=activ))#, kernel_initializer=kern_init, kernel_regularizer=l2(reg)))
    #model.add(Dropout(rate=0.2))

    model.add(Dense(units=64, activation=activ))#, kernel_initializer=kern_init, kernel_regularizer=l2(reg)))
    #model.add(Dropout(rate=0.2))
    
    #model.add(Dense(units=32, activation='elu'))#, kernel_initializer=kern_init, kernel_regularizer=l2(reg)))
    #model.add(Dropout(rate=0.2))
    
    model.add(Dense(units=16, activation=activ))#, kernel_initializer=kern_init, kernel_regularizer=l2(reg)))
    #model.add(Dropout(rate=0.2))
    
    model.add(Dense(units=1, activation=activ))#, kernel_initializer=kern_init, kernel_regularizer=l2(reg)))
    
    if custom_opt:
        opt = Adam(learning_rate=learning_rate, epsilon=epsilon) # was 1e-3    was 3e-5
    else:
        opt = 'adam'
    print(opt)
    model.compile(optimizer=opt, loss=loss)
    
    return model

In [ ]:
def a_print_fn(x):
    global train_name, model_structure
    dic = {'lambd':'L', 'conv2':'C', 'max_p':'P', 'flatt':'F', 'dense':'De', 'dropo':'D'}
    dic2 = {'C':4, 'De':0}
    dic3 = {',':0, ')':1, ' ':0}
    if x[:5] in dic:
        symbol = dic[x[:5]]
        symbol_unit = ''
        index = 13
        commaSpace_count = 0
        if symbol in dic2:
            while True:
                symbol_num = x[index]
                if symbol_num not in dic3:
                    if commaSpace_count == dic2[symbol]:
                        symbol_unit = symbol_unit + symbol_num
                    index = index + 1
                else:
                    if commaSpace_count == dic2[symbol]:
                        break
                    commaSpace_count = commaSpace_count+1
                    index = index + 1
        train_name = train_name + '-' + symbol + symbol_unit
#         print(train_name)
#     print(x)
    model_structure = np.append(model_structure, x)

In [ ]:
def setGlobal(model_name, batch_size, epoch_num, steps_per_epoch, model):
    global train_path, model_train_meta_loc, model_train_meta_loc2, model_structure, train_name
    global model_test_meta_loc, model_test_meta_loc2, test_result_path, test_result_path2
    global history_loc, weights_loc, weights_loc2, weights_loc3, activ
    # model_name = 'keras'
    #model_name = 'grayClipped'
    #batch_size = 40
    #epoch_num = 40
    #steps_per_epoch = 400

    # asset_path = '../train/assets'
    # train_name = f'model={model_name}_elu_C40_C20_De128_De64_De15_De1-batch_size={batch_size}-num_epoch={epoch_num}-steps_per_epoch={steps_per_epoch}'

    model_structure = np.empty((0))
    asset_path = '../train/assets/models2'
    train_name = f'model={model_name}-{activ}'
    model.summary(line_length=100, positions=[0.06,0.55,0.67,0.0], print_fn=a_print_fn)

    train_name_end = f'-B={batch_size}-E={epoch_num}-SP={steps_per_epoch}-R='
    # Need to manually change the ending.
    #train_name2 = train_name + train_name_end2 + '-5'
    train_name = train_name + train_name_end + 'NA'

    train_path = os.path.join(asset_path, train_name)
    #train_path2 = os.path.join(asset_path, train_name2)

    if not os.path.exists(train_path):
        os.makedirs(train_path)
        model_structure_loc = os.path.join(train_path, 'model_structure.txt')
        model_structure.tofile(model_structure_loc,sep="\n",format="%s")

    model_train_meta_loc = os.path.join(train_path, 'train_meta.csv')
    model_test_meta_loc = os.path.join(train_path, 'test_meta.csv')
    test_result_path = os.path.join(train_path, 'test.txt')
    model_train_meta_loc2 = os.path.join(train_path, 'train_meta2.csv')
    model_test_meta_loc2 = os.path.join(train_path, 'test_meta2.csv')
    test_result_path2 = os.path.join(train_path, 'test2.txt')

    history_loc = os.path.join(train_path, 'history.p')
    weights_loc = os.path.join(train_path, 'weights.h5')
    weights_loc2 = os.path.join(train_path, 'weights2.h5')
    
    #weights_loc3 = os.path.join(train_path2, 'weights2.h5')

    # index_count = np.zeros((20399), dtype='int16')

In [ ]:
def start(model_name, batch_size, epoch_num, steps_per_epoch, initial_epoch, load_weights, onlySet_global):
    global train_meta_loc, test_meta_loc, train_path
    
    model = createCNNModel()
    setGlobal(model_name, batch_size, epoch_num, steps_per_epoch, model)
    if load_weights:
        model.load_weights(weights_loc3)
    
    train_meta_loc = '../train/text/CSV'
    train_meta_loc = os.path.join(train_meta_loc, 'trainGrayClipped_meta.csv')
    test_meta_loc = '../test/text/CSV'
    test_meta_loc = os.path.join(test_meta_loc, 'testGrayClipped_meta.csv')
    
    if onlySet_global:
        return 0
    
    train_meta = pd.read_csv(train_meta_loc)
    train_meta_length = train_meta.shape[0]
    test_meta = pd.read_csv(test_meta_loc)
    test_meta_length = test_meta.shape[0]
    
    # Adding instances of zero speed from test.mp4
    stop_index_array = np.empty((0), dtype='int16')
    stop_index_array = np.append(stop_index_array, np.arange(1080,1640), axis=0)
    stop_index_array = np.append(stop_index_array, np.arange(6150,6200), axis=0)
    stop_index_array = np.append(stop_index_array, np.arange(6650,6700), axis=0)
    stop_index_array = np.append(stop_index_array, np.arange(7400,7430), axis=0)
    stop_index_array = np.append(stop_index_array, np.arange(8230,8330), axis=0)
    stop_index_array = np.append(stop_index_array, np.arange(8920,8945), axis=0)
    
    for idx, i in enumerate(stop_index_array):
        train_meta.loc[train_meta_length+idx] = test_meta.loc[i]
    #print('CSV len with stop:', train_meta.shape[0])
    train_meta.loc[train_meta_length:, 'speed'] = 0
    
    #data_index = np.loadtxt('../train/text/train_val_index.txt', np.dtype('int16'), delimiter='\n')
    data_index = np.arange(1, train_meta.shape[0])
    data_index[0] = (data_index[0] + 1)
    data_index[20399] = (data_index[20399] + 2)
    data_index[20400] = (data_index[20400] + 2)
    rng_state = np.random.get_state(1)
    np.random.shuffle(data_index)
    dataset_frames_num = data_index.shape[0]
    train_frames_num = int(.8*(dataset_frames_num))
    val_frames_num = int(.5*(dataset_frames_num - train_frames_num))
    #print('Train len:', train_frames_num, 'Val len:', val_frames_num)
    
    
    train_index = data_index[:train_frames_num]
    val_index = data_index[train_frames_num:(train_frames_num+val_frames_num)]
    #print('Train Index len:', train_index.shape, 'Val Index len:', val_index.shape)
    
    if not os.path.exists(train_path):
        os.makedirs(train_path)
    
    valid_generator = getValData(train_meta, val_index, True)
    val_size = len(val_index)
    train_size = len(train_index)
    train_generator = getTrainingData(train_meta, train_index, batch_size)
    
    earlyStopping = EarlyStopping(monitor='val_loss', 
                                  patience=1, 
                                  verbose=1, 
                                  min_delta = 0.23,
                                  mode='min',)
    modelCheckpoint = ModelCheckpoint(weights_loc, 
                                      monitor = 'val_loss', 
                                      save_best_only = True, 
                                      mode = 'min', 
                                      verbose = 1,
                                     save_weights_only = True)
    modelCheckpoint2 = ModelCheckpoint(weights_loc2, 
                                      monitor = 'loss', 
                                      save_best_only = True, 
                                      mode = 'min', 
                                      verbose = 1,
                                     save_weights_only = True)
    
    callbacks_list = [modelCheckpoint, modelCheckpoint2]
    
    history = model.fit_generator(
            train_generator, 
            steps_per_epoch = steps_per_epoch, 
            epochs = epoch_num,
            #initial_epoch = initial_epoch,
            callbacks = callbacks_list,
            verbose = 1,
            validation_data = valid_generator,
            validation_steps = val_size)

    pickle.dump(history.history, open(history_loc, "wb"))
    
    return history.history

In [ ]:
num_layers = 1
activ = 'elu'
reg = 0
learning_rate = 1e-3
epsilon = 1e-7
loss = 'mse'

kern_init = 'zeros'
reg2 = 0
pad = 'valid'
custom_opt = 1

In [ ]:
num_layers = 1
activ = 'elu'
reg = 0
learning_rate = 1e-3
epsilon = 1e-8
loss = 'mse'

kern_init = 'zeros'
reg2 = 0
pad = 'valid'
custom_opt = 1

            #     model_name    B   E  SP  IE   LW     GO
history = start('grayClipped', 40, 10, 400, 0, False, False)

In [ ]:
num_layers = 1
activ = 'elu'
reg = 0
learning_rate = 1e-4
epsilon = 1e-8
loss = 'mse'

kern_init = 'zeros'
reg2 = 0
pad = 'valid'
custom_opt = 1

            #     model_name    B   E  SP  IE   LW     GO
history = start('grayClipped', 40, 10, 400, 0, False, False)

In [ ]:
num_layers = 1
activ = 'elu'
reg = 0
learning_rate = 1e-5
epsilon = 1e-8
loss = 'mse'

kern_init = 'zeros'
reg2 = 0
pad = 'valid'
custom_opt = 1

            #     model_name    B   E  SP  IE   LW     GO
history = start('grayClipped', 40, 10, 400, 0, False, False)

In [ ]:
            #     model_name    B   E  SP  IE   LW     GO
history = start('grayClipped', 40, 10, 100, 0, False, False)

In [ ]:
            #     model_name    B   E  SP  IE   LW     GO
history = start('grayClipped', 40, 10, 400, 0, False, False)

In [ ]:
            #     model_name    B   E  SP  IE   LW     GO
history = start('grayClipped', 40, 10, 400, 0, False, False)

In [ ]:
history = pickle.load(open(history_loc, "rb" ))
print(history['loss'])
plt.plot(history['loss'], 'ro')
plt.plot(history['val_loss'], 'go')
plt.grid(which='both', axis='both')

In [ ]:
def predictSpeed(a_meta, a_meta_loc, a_weights_loc, getSpeed, setError):
    global model_train_meta_loc, model_test_meta_loc, batch_size

    dataset_frames_num = a_meta.shape[0]
    print(dataset_frames_num)
    
    data_index = np.arange(1, (dataset_frames_num))
    data_length = data_index.shape[0]
    print(data_length)
    
    data_gen = getValData(a_meta, data_index, getSpeed)
    
    model = createCNNModel()
    model.load_weights(a_weights_loc)
    
    predicted_speed = model.predict(data_gen, steps=data_length, verbose=1)
    predicted_speed.shape = dataset_frames_num-1
    
    a_meta = a_meta.assign(predicted_speed=pd.Series(np.empty((dataset_frames_num))).values)
    a_meta.loc[:, 'predicted_speed'] = np.nan
    a_meta.loc[1:,'predicted_speed'] = predicted_speed
    
    if setError:
        a_meta = a_meta.assign(error=pd.Series(np.empty((dataset_frames_num))).values)
        a_meta.loc[:, 'error'] = np.nan
        a_meta.loc[1:,'error'] = abs(a_meta.loc[1:, 'speed'] - predicted_speed)

    a_meta.to_csv(a_meta_loc, index=False)

In [ ]:
def rolling_median(a_meta, a_meta_loc, window_size, is_train):
    a_meta_length = a_meta.shape[0]
    is_odd = window_size%2
    beg_missing_range = int(window_size/2) if is_odd else int(window_size/2)
    end_missing_range = a_meta_length-beg_missing_range if is_odd else a_meta_length-(beg_missing_range-1)
    a_meta.loc[:, f'{window_size}med_prd_spd'] = np.nan
    a_meta.loc[1:,f'{window_size}med_prd_spd'] = a_meta[1:]['predicted_speed'].rolling(window_size, center=True).median()
    a_meta.loc[1:beg_missing_range,f'{window_size}med_prd_spd']=a_meta.loc[beg_missing_range+1, f'{window_size}med_prd_spd'] # was 25 50
    a_meta.loc[end_missing_range:,f'{window_size}med_prd_spd']=a_meta.loc[end_missing_range-1, f'{window_size}med_prd_spd'] # was 20376 20351

    if is_train:
        a_meta.loc[:][f'{window_size}med_error'] = np.nan
        a_meta.loc[1:,f'{window_size}med_error'] = abs(a_meta.loc[1:, 'speed'] - a_meta.loc[1:,f'{window_size}med_prd_spd'])
    a_meta.to_csv(a_meta_loc, index=False)
    print(is_odd, beg_missing_range, end_missing_range)

In [ ]:
def stopSignSpeed(a_meta):
    stop_length = a_meta.loc[1080:1640].shape[0]
    test1 = 0
    test2 = 0
    test3 = 0
    for i in a_meta.loc[1080:1640, 'predicted_speed']:
        test1 = test1 + i
    test1_loss = test1/stop_length
    for i in a_meta.loc[1080:1640, '50med_prd_spd']:
        test2 = test2 + i
    test2_loss = test2/stop_length
    for i in a_meta.loc[1080:1640, '100med_prd_spd']:
        test3 = test3 + i
    test3_loss = test3/stop_length
    
    print(test1_loss, test2_loss, test3_loss)
def printMSE(a_meta, is_train, start=1, stop=None):
    if is_train:
        data_index = np.arange(1, a_meta.shape[0])
        rng_state = np.random.get_state(1)
        np.random.shuffle(data_index)
        dataset_frames_num = data_index.shape[0]
        train_frames_num = int(.8*(dataset_frames_num))
        val_frames_num = int(.5*(dataset_frames_num - train_frames_num))
        test_frames_num = int((dataset_frames_num - (train_frames_num+val_frames_num)))
        print('Train len:', train_frames_num, 'Val len:', val_frames_num, 'Test len:', test_frames_num)


        train_index = data_index[:train_frames_num]
        val_index = data_index[train_frames_num:(train_frames_num+val_frames_num)]
        test_index = data_index[(train_frames_num+val_frames_num):]
        
#         start = 1 if start is None else start
        stop = a_meta.shape[0]-1 if stop is None else stop
        print(mean_squared_error(a_meta.loc[train_index, 'speed'], a_meta.loc[train_index, 'predicted_speed']),
             mean_squared_error(a_meta.loc[val_index, 'speed'], a_meta.loc[val_index, 'predicted_speed']),
             mean_squared_error(a_meta.loc[test_index, 'speed'], a_meta.loc[test_index, 'predicted_speed']))
        
        print(mean_squared_error(a_meta.loc[start:stop, 'speed'], a_meta.loc[start:stop, '50med_prd_spd']))
        print(mean_squared_error(a_meta.loc[start:stop, 'speed'], a_meta.loc[start:stop, '100med_prd_spd']))
    else:
        print(mean_squared_error(np.zeros((1641-1080)), a_meta.loc[1080:1640, 'predicted_speed']))
        print(mean_squared_error(np.zeros((1641-1080)), a_meta.loc[1080:1640, '50med_prd_spd']))
        print(mean_squared_error(np.zeros((1641-1080)), a_meta.loc[1080:1640, '100med_prd_spd']))
def printMeta(a_meta, start=0, stop=None):
#     start = 0 if start is None else start
    stop = a_meta.shape[0] if stop is None else stop
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(a_meta[start:stop])    

In [ ]:
train_meta1 = pd.read_csv(train_meta_loc)
predictSpeed(train_meta1, model_train_meta_loc, weights_loc, False, True)

train_meta2 = pd.read_csv(train_meta_loc)
predictSpeed(train_meta2, model_train_meta_loc2, weights_loc2, False, True)

In [ ]:
train_meta1 = pd.read_csv(model_train_meta_loc)
rolling_median(train_meta1, model_train_meta_loc, 50, True)
rolling_median(train_meta1, model_train_meta_loc, 100, True)

train_meta2 = pd.read_csv(model_train_meta_loc2)
rolling_median(train_meta2, model_train_meta_loc2, 50, True)
rolling_median(train_meta2, model_train_meta_loc2, 100, True)

In [ ]:
train_meta1 = pd.read_csv(model_train_meta_loc)
printMSE(train_meta1, True)
print()
train_meta2 = pd.read_csv(model_train_meta_loc2)
printMSE(train_meta2, True)

In [ ]:
test_meta1 = pd.read_csv(test_meta_loc)
predictSpeed(test_meta1, model_test_meta_loc, weights_loc, False, False)

test_meta2 = pd.read_csv(test_meta_loc)
predictSpeed(test_meta2, model_test_meta_loc2, weights_loc2, False, False)

In [ ]:
test_meta1 = pd.read_csv(model_test_meta_loc)
rolling_median(test_meta1, model_test_meta_loc, 50, False)
rolling_median(test_meta1, model_test_meta_loc, 100, False)

test_meta2 = pd.read_csv(model_test_meta_loc2)
rolling_median(test_meta2, model_test_meta_loc2, 50, False)
rolling_median(test_meta2, model_test_meta_loc2, 100, False)

In [ ]:
test_meta1 = pd.read_csv(model_test_meta_loc)
stopSignSpeed(test_meta1)
print()
test_meta2 = pd.read_csv(model_test_meta_loc2)
stopSignSpeed(test_meta2)

In [ ]:
printMSE(test_meta1, False)
print()
printMSE(test_meta2, False)

In [ ]:
test_result = test_meta1.loc[:, '100med_prd_spd']
test_result.to_csv(test_result_path, index=False)

test_result = test_meta2.loc[:, '100med_prd_spd']
test_result.to_csv(test_result_path2, index=False)

In [ ]:
train_meta1 = pd.read_csv(model_train_meta_loc)
train_meta2 = pd.read_csv(model_train_meta_loc2)

In [ ]:
test_meta1 = pd.read_csv(model_test_meta_loc)
test_meta2 = pd.read_csv(model_test_meta_loc2)

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
print(fig_size)
fig_size[0] = 15
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size
fig_size2 = plt.rcParams["figure.figsize"]
print(fig_size2)

In [ ]:
def plotSpeed(a_meta, is_train, saveFile, speed_type1, speed_type2=None):
    global train_path
    fig, ax = plt.subplots()
    if speed_type2 is not None:
        ax.plot(a_meta.loc[1:, 'image_index'], a_meta.loc[1:, speed_type2], 'r', linewidth=2)
    if speed_type1 is not None:
        ax.plot(a_meta.loc[1:, 'image_index'], a_meta.loc[1:, speed_type1], 'bo', markersize=2)
    ax.grid(which='both', axis='both')
    ax.xaxis.set_minor_locator(MultipleLocator(500))
    ax.yaxis.set_minor_locator(MultipleLocator(1))
    if is_train:
        randomNumber = plt.xticks(np.arange(0, 20400, 2000))
        randomNumber = plt.yticks(np.arange(0, 32, 2))
    else:
        randomNumber = plt.yticks(np.arange(0, 38, 2))
        randomNumber = plt.xticks(np.arange(0, 10798, 1000))
    if saveFile:
        plotPath = os.path.join(train_path, speed_type1+'.png')
        fig.savefig(plotPath, bbox_inches='tight', pad_inches=0.1)

In [ ]:
plotSpeed(train_meta1, True, False, None, 'speed')

In [ ]:
plotSpeed(train_meta2, True, False, None, 'speed')

In [ ]:
plotSpeed(train_meta1, True, False, 'predicted_speed')

In [ ]:
plotSpeed(train_meta2, True, False, 'predicted_speed')

In [ ]:
plotSpeed(train_meta1, True, False, 'predicted_speed', 'speed')

In [ ]:
plotSpeed(train_meta2, True, False, 'predicted_speed', 'speed')

In [ ]:
plotSpeed(train_meta1, True, False, '50med_prd_spd', 'speed')

In [ ]:
plotSpeed(train_meta2, True, False, '50med_prd_spd', 'speed')

In [ ]:
plotSpeed(train_meta1, True, False, '100med_prd_spd', 'speed')

In [ ]:
plotSpeed(train_meta2, True, False, '100med_prd_spd', 'speed')

In [ ]:
plotSpeed(test_meta1, False, False, '50med_prd_spd')

In [ ]:
plotSpeed(test_meta2, False, False, '50med_prd_spd')

In [ ]:
plotSpeed(test_meta1, False, False, '100med_prd_spd')

In [ ]:
plotSpeed(test_meta2, False, True, '100med_prd_spd')

In [ ]:
plotSpeed(test_meta1, False, False, 'predicted_speed')

In [ ]:
plotSpeed(test_meta2, False, False, 'predicted_speed')

In [ ]:
reallyOldPath = '../train/assets/model=keras_elu_C40_C20_De128_De64_De15_De1-batch_size=40-num_epoch=20-steps_per_epoch=400'
a_meta_loc = os.path.join(reallyOldPath, 'test_meta.csv')
a_meta=pd.read_csv(a_meta_loc)
fig, ax = plt.subplots()
ax.plot(a_meta.loc[1:, 'image_index'], a_meta.loc[1:, 'med_prd_spd'], 'bo', markersize=2)
ax.grid(which='both', axis='both')
ax.xaxis.set_minor_locator(MultipleLocator(500))
ax.yaxis.set_minor_locator(MultipleLocator(1))
randomNumber = plt.yticks(np.arange(0, 38, 2))
randomNumber = plt.xticks(np.arange(0, 10798, 1000))

In [ ]:
randomArray = np.loadtxt('../adeebsTest.txt', np.dtype('float32'), delimiter='\n')
fig, ax = plt.subplots()
ax.plot(randomArray[:], 'b', linewidth=2)
ax.plot(test_meta2.loc[1:, 'image_index'], test_meta2.loc[1:, '100med_prd_spd'], 'r', linewidth=2)#, markersize=2)
ax.grid(which='both', axis='both')
ax.xaxis.set_minor_locator(MultipleLocator(500))
ax.yaxis.set_minor_locator(MultipleLocator(1))
randomNumber = plt.yticks(np.arange(0, 38, 2))
randomNumber = plt.xticks(np.arange(0, 10798, 1000))
plot_path = os.path.join(train_path, '100med_prd_spd-adeeb'+'.png')
fig.savefig(plot_path, bbox_inches='tight', pad_inches=0.1)

In [ ]:
randomArray = np.loadtxt('../jovsaTest.txt', np.dtype('float32'), delimiter='\n')
print(randomArray.shape)
fig, ax = plt.subplots()
ax.plot(randomArray[:], 'bo', markersize=2)
ax.grid(which='both', axis='both')
ax.xaxis.set_minor_locator(MultipleLocator(500))
ax.yaxis.set_minor_locator(MultipleLocator(1))
randomNumber = plt.yticks(np.arange(0, 25, 2))
randomNumber = plt.xticks(np.arange(0, 10798, 1000))

In [ ]:
randomArray = np.loadtxt('../experienTest1.txt', np.dtype('float32'), delimiter='\n')
print(randomArray.shape)
fig, ax = plt.subplots()
ax.plot(randomArray[:], 'b')#, markersize=5)
ax.grid(which='both', axis='both')
ax.xaxis.set_minor_locator(MultipleLocator(500))
ax.yaxis.set_minor_locator(MultipleLocator(1))
randomNumber = plt.yticks(np.arange(0, 26, 2))
randomNumber = plt.xticks(np.arange(0, 10798, 1000))